In [1]:
import glob
import os
import shutil
from tqdm import tqdm
import pandas as pd

In [2]:
# this is to set up the dataset of the Wortschatzinsel project in a way that I can use it with my neural networks in a better way. It is built to run on phobos. And then used to copy it over to the hlrn because i have not found a way
# to simply sabamount (gio) to hlrn may be possible in future
base_path = "/run/user/715136/gvfs/smb-share:server=wfs-biologie.top.gwdg.de,share=ubps-all$/Language_Acquisition/HIWI/HIWIstudies/OLDstudiesHIWI/SmallFaces in the Wild HIWI/Data/"
neon_scene_views_paths = glob.glob(os.path.join(base_path,"Neon/*/4*/*/*Scene*.mp4"))
pupilcore_scene_views_paths = glob.glob(os.path.join(base_path,"Pupil Core/raspi-*/*/*/world.mp4"))
pupilcore_scene_views_alternative_paths = glob.glob(os.path.join(base_path,"Pupil Core/raspi-*/*/*/*/world.mp4"))

In [3]:

exclusions = ['ignore', 'missing', 'invalid', 'test']
neon_scene_views_paths = [path for path in neon_scene_views_paths if not any(sub in path for sub in exclusions)]
neon_scene_view_nrs = [int(path.split("/")[-3]) for path in neon_scene_views_paths]
neon_scene_vies_old_name = [path.split("/")[-3] for path in neon_scene_views_paths]

pupilcore_scene_views_paths = [path for path in pupilcore_scene_views_paths if not any(sub in path for sub in exclusions)]
pupilcore_scene_view_nrs = [int(path.split("/")[-2].split("_")[0]) for path in pupilcore_scene_views_paths]
pupilcore_scene_view_old_name = [path.split("/")[-2] for path in pupilcore_scene_views_paths]
exclusions.append("old")
pupilcore_scene_views_alternative_paths = [path for path in pupilcore_scene_views_alternative_paths if not any(sub in path for sub in exclusions)]
pupilcore_scene_view_alternative_nrs = [int(path.split("/")[-3].split("_")[0]) for path in pupilcore_scene_views_alternative_paths]
pupilcore_scene_view_alternative_old_name = [path.split("/")[-3] for path in pupilcore_scene_views_alternative_paths]

scene_view_paths = neon_scene_views_paths + pupilcore_scene_views_paths + pupilcore_scene_views_alternative_paths
scene_view_nrs = neon_scene_view_nrs + pupilcore_scene_view_nrs + pupilcore_scene_view_alternative_nrs
scene_view_old_name = neon_scene_vies_old_name + pupilcore_scene_view_old_name + pupilcore_scene_view_alternative_old_name

scene_view_df = pd.DataFrame({"path":scene_view_paths,"scene_view_nr":scene_view_nrs, "old_name":scene_view_old_name})
scene_view_df["neon"] = scene_view_df["path"].str.contains("/Neon/")

scene_view_df["nr"] = range(len(scene_view_df))

scene_view_df["new_name"] = scene_view_df["scene_view_nr"].astype(str) + "_help_" +  scene_view_df["nr"].astype(str)

In [4]:
#because naming changed after some stuff happend on the server, therefore we need to make sure that the names are still correct next time
# 

#scene_view_df = pd.read_csv("/usr/users/vhassle/datasets/Wortschatzinsel/head_mounted_data/scene_view_creation_df.csv")
scene_view_orig_df = pd.read_csv("/usr/users/vhassle/datasets/Wortschatzinsel/head_mounted_data/scene_view_creation_df_orig_names.csv")
scene_view_orig_df.columns = [name+"_orig" for name in scene_view_orig_df.columns]
scene_view_df = pd.merge(scene_view_df, scene_view_orig_df, left_on="old_name", right_on="old_name_orig", how="inner")
scene_view_df["new_name"] = scene_view_df["new_name_orig"].combine_first(scene_view_df["new_name"])
scene_view_df.drop(columns=[name for name in scene_view_df.columns if "_orig" in name], inplace=True)

scene_view_df_neon = scene_view_df[scene_view_df["neon"]]
scene_view_df_pupilcore = scene_view_df[~scene_view_df["neon"]]

base_path_server = "/usr/users/vhassle/datasets/Wortschatzinsel/head_mounted_data"

In [5]:
# copies over all the video files that takes a while therefore it is commented out as default
for index, row in tqdm(scene_view_df.iterrows(), desc="Copying files"):
    source_path = row['path']
    destination_path = f"{base_path_server}/videos/{row['new_name']}.mp4"
    
    # Skip if destination file already exists
    if not os.path.exists(destination_path):
        shutil.copy(source_path, destination_path)
        print(f"Copied: {source_path} -> {destination_path}")
    else:
        print(f"Skipping: Destination file exists - {destination_path}")

Copying files: 105it [00:00, 1036.98it/s]

Skipping: Destination file exists - /usr/users/vhassle/datasets/Wortschatzinsel/head_mounted_data/videos/437_help_0.mp4
Skipping: Destination file exists - /usr/users/vhassle/datasets/Wortschatzinsel/head_mounted_data/videos/435_help_1.mp4
Skipping: Destination file exists - /usr/users/vhassle/datasets/Wortschatzinsel/head_mounted_data/videos/436_help_2.mp4
Skipping: Destination file exists - /usr/users/vhassle/datasets/Wortschatzinsel/head_mounted_data/videos/442_help_3.mp4
Skipping: Destination file exists - /usr/users/vhassle/datasets/Wortschatzinsel/head_mounted_data/videos/449_help_4.mp4
Skipping: Destination file exists - /usr/users/vhassle/datasets/Wortschatzinsel/head_mounted_data/videos/441_help_5.mp4
Skipping: Destination file exists - /usr/users/vhassle/datasets/Wortschatzinsel/head_mounted_data/videos/439_help_6.mp4
Skipping: Destination file exists - /usr/users/vhassle/datasets/Wortschatzinsel/head_mounted_data/videos/443_help_7.mp4
Skipping: Destination file exists - /usr

In [6]:
# copy the annotations of the eye tracking data
for index, row in tqdm(scene_view_df_neon.iterrows()):
    directory_path = os.path.dirname(row["path"])
    gaze_files = glob.glob(os.path.join(directory_path, "neon_player/exports/*/gaze_positions.csv"))
    gaze_path = gaze_files[0] if gaze_files else None
    scene_view_df_neon.at[index, "gaze_position_path"] = gaze_path

    folder_annotation_path = f"{base_path_server}/eye_tracking_annotations/{row['new_name']}"
    os.makedirs(folder_annotation_path, exist_ok=True)
    dest_file = os.path.join(folder_annotation_path, "gaze_positions.csv")
    if os.path.exists(dest_file):
        continue
    # if gaze_path is not None:
    #     shutil.copy(gaze_path, dest_file)
    world_timestamps_files = glob.glob(os.path.join(directory_path, "neon_player/exports/*/world_timestamps.csv"))
    world_timestamps_path = world_timestamps_files[0] if world_timestamps_files else None
    scene_view_df_neon.at[index, "world_timestamps_path"] = world_timestamps_path
    # if world_timestamps_path is not None:
    #     shutil.copy(world_timestamps_path, os.path.join(folder_annotation_path, "world_timestamps.csv"))


0it [00:00, ?it/s]/tmp/ipykernel_12564/3223248934.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  scene_view_df_neon.at[index, "gaze_position_path"] = gaze_path
13it [00:00, 18.46it/s]/tmp/ipykernel_12564/3223248934.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  scene_view_df_neon.at[index, "world_timestamps_path"] = world_timestamps_path
43it [00:02, 15.59it/s]


In [7]:
# processing this for the pupilcore data
example_path = os.path.dirname(scene_view_df_pupilcore["path"][50])
glob.glob(os.path.join(example_path, "exports/*/gaze_positions.csv"))


for index, row in tqdm(scene_view_df_pupilcore.iterrows()):
    directory_path = os.path.dirname(row["path"])
    gaze_files = glob.glob(os.path.join(directory_path, "exports/*/gaze_positions.csv"))
    gaze_path = gaze_files[0] if gaze_files else None
    scene_view_df_pupilcore.at[index, "gaze_position_path"] = gaze_path

    folder_annotation_path = f"{base_path_server}/eye_tracking_annotations/{row['new_name']}"
    dest_file = os.path.join(folder_annotation_path, "gaze_positions.csv")
    if os.path.exists(dest_file):
        continue
    os.makedirs(folder_annotation_path, exist_ok=True)
    
    if gaze_path is not None:
        shutil.copy(gaze_path, os.path.join(folder_annotation_path, "gaze_positions.csv"))
        # timesteps is not reuired in this case

0it [00:00, ?it/s]/tmp/ipykernel_12564/2206283943.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  scene_view_df_pupilcore.at[index, "gaze_position_path"] = gaze_path
62it [00:05, 10.97it/s]


In [ ]:
csv_filename = f"{base_path_server}/scene_view_creation_df.csv"
scene_view_df.to_csv(csv_filename, index=False)
print(f"Descriptive dataframe saved to {csv_filename}")

# rsync -avz vhassle@phobos.cidas.uni-goettingen.de:"/usr/users/vhassle/datasets/Wortschatzinsel/head_mounted_data/videos/" "/mnt/lustre-emmy-hdd/usr/u11216/datasets/Wortschatzinsel/eye_tracking/head_mounted_data/videos/"
# rsync -avz vhassle@phobos.cidas.uni-goettingen.de:"/usr/users/vhassle/datasets/Wortschatzinsel/head_mounted_data/eye_tracking_annotations/" "/mnt/lustre-emmy-hdd/usr/u11216/datasets/Wortschatzinsel/eye_tracking/head_mounted_data/eye_tracking_annotations/"

# scp -avz "/mnt/lustre-emmy-hdd/usr/u11216/outputs/age_gender_combined_training_r02_25v2/output_with_eyetracking" vhassle@phobos.cidas.uni-goettingen.de:"/run/user/715136/gvfs/smb-share:server=wfs-biologie.top.gwdg.de,share=ubps-all\$/Language_Acquisition/HIWI/HIWIstudies/OLDstudiesHIWI/SmallFaces in the Wild HIWI/Data/head_mounted_tracking_results/results_data"

Descriptive dataframe saved to /usr/users/vhassle/datasets/Wortschatzinsel/head_mounted_data/scene_view_creation_df.csv


In [ ]:
#scp -rv "/mnt/lustre-emmy-hdd/usr/u11216/datasets/Wortschatzinsel/object_detection_train/annotations/videos" vhassle@phobos.cidas.uni-goettingen.de:"/run/user/715136/gvfs/smb-share:server=wfs-biologie.top.gwdg.de,share=ubps-all\$/Language_Acquisition/HIWI/HIWIstudies/OLDstudiesHIWI/SmallFaces in the Wild HIWI/Data/top_view_tracking_results/videos"